In [1]:
import os 
os.chdir("../..")

import pandas as pd
from task1.config import ProjectPaths
import requests

paths = ProjectPaths()
df = pd.read_csv(paths.english_data_dir / "train_en.tsv", sep="\t")[:100]



In [2]:
### ZERO SHOT ###

instruction_prompt = """You will receive a set of sentences that are either subjective or objective.
Your task is to classify each sentence as either subjective or objective. The format you will receive the sentences is: 

\n\n\n

sentence: <sentence> \n
sentence: <sentence> \n
sentence: <sentence> \n
...
sentence: <sentence> \n
\n\n\n

Your answer should be OBJ if the sentence is objective and SUBJ if the sentence is subjective. The format you should follow for your answer is: 

<answer> \n
<answer> \n
<answer> \n
...
<answer> \n

Do not provide any other information than the answer, i just need the answers in the format provided above. Keep track of the number of sentences, do not provide more answers than the number of sentences. Following the instruction provided above, these are the sentences: 
\n\n\n

"""

In [3]:
def create_prompt(df):
    prompt = ""
    for _ , row in df.iterrows():
        prompt += f"sentence: {row['sentence']} \n"
    return prompt + "\nWrite your answer below: \n"

In [ ]:
import re

def parse_response(response_text):
    """
    Removes the content between <think> and </think> tags from a string.
    """
    # This regular expression finds and removes content between <think> and </think> tags.
    # re.DOTALL ensures that '.' matches newline characters as well,
    # so multi-line <think> blocks are correctly handled.
    cleaned_text = re.sub(r"<think>.*?</think>", "", response_text, flags=re.DOTALL).split("\n")
    # .strip() removes any leading/trailing whitespace from the final string.
    answers =[text.strip() for text in cleaned_text if text.strip()]
    print(answers)
    answers = [0 if answer == "OBJ" else 1 for answer in answers]
    return answers

: 

In [ ]:
def get_response(df):
    y_pred = []
    for enum, i in enumerate(range(0, len(df), 10)):
        one_shot_prompt = instruction_prompt + create_prompt(df[i:i+10])
        response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "deepseek-r1:7b",
            "prompt": one_shot_prompt,
            "stream": False, 
            "temperature": 0.0, 
            "top_p": 0.9,
        }).json()
        y_pred_batch = parse_response(response["response"])
        y_pred.extend(y_pred_batch)
        print(f"Batch {enum+1} completed, {len(y_pred_batch)} predictions made for {len(df[i:i+10])} sentences")
    return y_pred

y_pred = get_response(df)
len(y_pred)



['OBJ', 'OBJ', 'SUBJ', 'SUBJ', 'OBJ', 'SUBJ', 'OBJ', 'SUBJ', 'OBJ', 'OBJ']
Batch 1 completed, 10 predictions made for 10 sentences
['OBJ', 'SUBJ', 'OBJ', 'OBJ', 'SUBJ', 'SUBJ', 'OBJ', 'SUBJ', 'OBJ', 'OBJ']
Batch 2 completed, 10 predictions made for 10 sentences
['OBJ', 'OBJ', 'OBJ', 'OBJ', 'OBJ', 'OBJ', 'OBJ', 'OBJ', 'SUBJ', 'OBJ']
Batch 3 completed, 10 predictions made for 10 sentences
['SUBJ', 'OBJ', 'OBJ', 'SUBJ', 'OBJ', 'SUBJ', 'OBJ', 'OBJ', 'SUBJ', 'OBJ']
Batch 4 completed, 10 predictions made for 10 sentences
['OBJ', 'Obj', 'Obj', 'Obj', 'Obj', 'Obj', 'Subj', 'Obj', 'Obj', 'Obj', 'Subj']
Batch 5 completed, 11 predictions made for 10 sentences


In [ ]:
y_true = df["label"].tolist()
y_true = [0 if label == "OBJ" else 1 for label in y_true]

In [52]:
from sklearn.metrics import f1_score

def compute_f1_macro(y_pred, y_true):
    return f1_score(y_true, y_pred, average="macro")

f1_macro = compute_f1_macro(y_pred, y_true)
print(f1_macro)






ValueError: Found input variables with inconsistent numbers of samples: [100, 101]